In [1]:
import os
from tqdm import tqdm
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings

In [2]:
embeddings = (
    OllamaEmbeddings(model="nomic-embed-text",base_url="http://localhost:11434")
)  

In [3]:
loader = PyPDFDirectoryLoader("./pdfs-master")
docs = loader.load()

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)   

print("Document splitting a in process")
docs = text_splitter.split_documents(docs)
print("Document splitted")
docs[1]

Document splitting a in process
Document splitted


Document(metadata={'source': 'pdfs-master\\NVIDIA - Accelerated Computing and the Democratization of Supercomputing Technical Overview (sc18-tesla-democratization-tech-overview-r4-web).pdf', 'page': 1}, page_content='ACCELERATED COMPUTING AND THE DEMOCRATIZATION OF SUPERCOMPUTING | TECHNICAL OVERVIEW | 2Table of Contents:\nThe Accelerated Data Center \nOptimizing Data Center Productivity\nSame Throughput with Fewer Server Nodes\nAren’t GPU-Accelerated Servers More Expensive? \nWhat About Operational Cost?\nWhat if the CPU is Given Away at No Cost?\nMaximizing Budget and Throughput\nLower Cost with  Acceleration\nDemocratizing the Supercomputer3\n6\n7\n8\n10\n11\n12\n13\n14')

In [16]:
from langchain_community.vectorstores import Milvus
MILVUS_PORT="19530"
MILVUS_HOST="127.0.0.1"
 
print("start connecting to Milvus")
vector_db = Milvus.from_documents(
    docs,
    embeddings,
    connection_args={"host": MILVUS_HOST ,"port": MILVUS_PORT,"db_name":"default"},
    collection_name="NvidiaPDF",
)

start connecting to Milvus


In [17]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

In [19]:
from redisvl.extensions.llmcache import SemanticCache
 
llmcache = SemanticCache(
    name="PDFs",
    prefix="PDFs",
    redis_url="redis://localhost:6379",
    distance_threshold=0.1
)
llmcache.set_threshold(0.1)

c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [20]:
!rvl index listall --host localhost --port 6379

15:20:25 [RedisVL] INFO   Indices:
15:20:25 [RedisVL] INFO   1. PDFs
